In [14]:
## Proyecto 2
## Carlos Rios - 100017078

# Creando el entorno de trabajo
# Importar librerías básicas
import pandas as pd
import numpy as np

# Visualización de los datos
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Librerías de la red neuronal (Keras)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout



In [16]:
# Cargar el conjunto de datos
# PASO 2 — Cargar dataset
df = pd.read_excel(r"C:\datos.xlsx")

# Ver primeras filas
df.head()


,no,tipoDeEntidadPadre,tipoEntidad,entidadCompradora,unidadCompradora,nogConcurso,descripción,modalidad,subModalidad,nit,...,añoDePublicación,fechaDeÚltimaAdjudicación,fechaDeAdjudicación,mesDeAdjudicación,añoDeAdjudicación,categorías,estatusDelConcurso,fechaCierreRecepciónOfertas,mesDeCierreRecepción,añoDeCierreRecepción
0,1,Sector Público,"Entidades Descentralizadas, Autónomas y de Seg...",INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,"HOSPITAL DE MALACATAN, SAN MARCOS",1325914,VERAPAMILO TABLETA 240 MG,Compra Directa con Oferta Electrónica (Art. 43...,NaN,93752180,...,2025,2025-08-09,2025-08-09,AGOSTO,2025,Salud e insumos hospitalarios,Adjudicado,2025-08-07,AGOSTO,2025
1,2,Sector Público,"Entidades Descentralizadas, Autónomas y de Seg...",INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,"HOSPITAL DE MALACATAN, SAN MARCOS",3252078,"SIMETICONA TABLETA, TABLETA MASTICABLE O CÁPSU...",Compra Directa con Oferta Electrónica (Art. 43...,NaN,8098921,...,2025,2025-08-13,2025-08-13,AGOSTO,2025,Salud e insumos hospitalarios,Adjudicado,2025-08-07,AGOSTO,2025
2,3,Sector Público,"Entidades Descentralizadas, Autónomas y de Seg...",INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,"HOSPITAL DE MALACATAN, SAN MARCOS",7194064,FENOBARBITAL SÓDICO SOLUCIÓN INYECTABLE 100 MG...,Compra Directa con Oferta Electrónica (Art. 43...,NaN,8163294,...,2025,2025-08-13,2025-08-13,AGOSTO,2025,Salud e insumos hospitalarios,Adjudicado,2025-08-05,AGOSTO,2025
3,4,Sector Público,"Gobiernos Locales (Municipalidades, Mancomunid...","MUNICIPALIDAD DE SAN JOSÉ PINULA, GUATEMALA",UNIDAD DE COMPRAS Y CONTRATACIONES,7291639,ADQUISICIÓN DE PÓLIZAS DE SEGURO PARA MOTOCICL...,Negociaciones entre Entidades Públicas (Art. 2...,NaN,330388,...,2025,2025-04-28,2025-04-28,ABRIL,2025,"Seguros, fianzas y servicios bancarios",Adjudicado,2025-04-21,ABRIL,2025
4,5,Sector Público,"Entidades Descentralizadas, Autónomas y de Seg...",INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,"HOSPITAL DE MALACATAN, SAN MARCOS",10031251,"SUPLEMENTO ALIMENTICIO, POLVO PARA SUSPENSIÓN ...",Compra Directa con Oferta Electrónica (Art. 43...,NaN,6177999,...,2025,2025-08-08,2025-08-08,AGOSTO,2025,Salud e insumos hospitalarios,Adjudicado,2025-08-06,AGOSTO,2025


In [31]:
# Construyendo el DF de trabajo
df.columns

# Clasificación de la columna "monto" en rangos: Bajo, Medio, Alto
q1 = df["monto"].quantile(0.25)
q3 = df["monto"].quantile(0.75)

def clasificar_monto(x):
    if x <= q1:
        return "Bajo"
    elif x <= q3:
        return "Medio"
    else:
        return "Alto"

df["montoRango"] = df["monto"].apply(clasificar_monto)

# Crear entGrp2 basado en tipoEntidad
def map_entgrp2(tipo):
    if tipo == "Administración Central":
        return "Administracion_Central"
    elif tipo == "Entidades Descentralizadas, Autónomas y de Seguridad Social":
        return "Descentralizada"
    elif tipo == "Empresas Públicas (Nacionales y Municipales)":
        return "Empresa_Publica"
    elif tipo == "Gobiernos Locales (Municipalidades, Mancomunidades, etc.)":
        return "Municipal"
    elif tipo == "ONG's, patronatos, comités, asociaciones y fundaciones":
        return "ONG"
    else:
        return "Otros"

df["entGrp2"] = df["tipoEntidad"].apply(map_entgrp2)

df["entidadCompradora"].drop_duplicates().head(40).tolist()
def map_entgrp(entidad):
    e = entidad.upper()

    # Banco de Guatemala
    if "BANCO DE GUATEMALA" in e:
        return "Banco_Guatemala"

    # Bomberos
    if "BOMBER" in e:  # cubre Bomberos Municipales y Voluntarios
        return "Bomberos"

    # Descentralizadas
    if (
        "IGSS" in e or
        "INDE" in e or
        "SAT" in e or
        "RENAP" in e or
        "INFOM" in e or
        "CIENCIAS FORENSES" in e or
        "INACIF" in e or
        "MIGRA" in e
    ):
        return "Descentralizada"

    # Obras Sociales
    if "OBRAS SOCIALES" in e:
        return "Obras_Sociales"

    # ONG, asociaciones, fundaciones
    if (
        "ASOCIACION" in e or
        "ASOCIACIÓN" in e or
        "FUNDACION" in e or
        "FUNDACIÓN" in e or
        "ONG" in e
    ):
        return "ONG_Asociacion"

    # Organismos deportivos
    if (
        "FEDERACION" in e or
        "FEDERACIÓN" in e or
        "CONFEDERACION" in e or
        "CONFEDERACIÓN" in e or
        "LIGA" in e
    ):
        return "Organismo_Deportivo"

    # Todo lo demás
    return "Otra"

# Crear la nueva columna entGrp
# Crear la nueva columna entGrp
df["entGrp"] = df["entidadCompradora"].apply(map_entgrp)

# Crear el df_reducido limpio y sin indentación incorrecta
df_reducido = df[[
    "entidadCompradora",
    "monto",
    "categorías",
    "tipoEntidad",
    "entGrp",
    "montoRango",
    "entGrp2"
]].copy()

# Ver primeras filas
df_reducido.head()



,entidadCompradora,monto,categorías,tipoEntidad,entGrp,montoRango,entGrp2
0,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,1987.50,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Bajo,Descentralizada
1,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,36000.00,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Medio,Descentralizada
2,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,293.70,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Bajo,Descentralizada
3,"MUNICIPALIDAD DE SAN JOSÉ PINULA, GUATEMALA",222223.47,"Seguros, fianzas y servicios bancarios","Gobiernos Locales (Municipalidades, Mancomunid...",Otra,Alto,Municipal
4,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,90000.00,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Alto,Descentralizada


In [32]:
# Codificar variables categóricas
from sklearn.preprocessing import LabelEncoder

# Creamos una copia para no modificar df_reducido directamente
df_nn = df_reducido.copy()

# Crear codificadores
le_tipoEntidad = LabelEncoder()
le_entGrp2 = LabelEncoder()
le_montoRango = LabelEncoder()

# Aplicar codificación
df_nn["tipoEntidad"] = le_tipoEntidad.fit_transform(df_nn["tipoEntidad"])
df_nn["entGrp2"] = le_entGrp2.fit_transform(df_nn["entGrp2"])
df_nn["montoRango"] = le_montoRango.fit_transform(df_nn["montoRango"])

df_nn.head()


,entidadCompradora,monto,categorías,tipoEntidad,entGrp,montoRango,entGrp2
0,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,1987.50,Salud e insumos hospitalarios,3,Descentralizada,1,1
1,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,36000.00,Salud e insumos hospitalarios,3,Descentralizada,2,1
2,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,293.70,Salud e insumos hospitalarios,3,Descentralizada,1,1
3,"MUNICIPALIDAD DE SAN JOSÉ PINULA, GUATEMALA",222223.47,"Seguros, fianzas y servicios bancarios",5,Otra,0,3
4,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,90000.00,Salud e insumos hospitalarios,3,Descentralizada,0,1


In [33]:
# PASO 5 — Separar variables predictoras y variable objetivo

X = df_nn[["tipoEntidad", "entGrp2"]]
y = df_nn["montoRango"]

X.head(), y.head()


(   tipoEntidad  entGrp2
 0            3        1
 1            3        1
 2            3        1
 3            5        3
 4            3        1,
 0    1
 1    2
 2    1
 3    0
 4    0
 Name: montoRango, dtype: int64)

In [34]:
# Definiendo el train y test sets
# PASO 6 — Dividir datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((166751, 2), (41688, 2))

In [35]:
# Normalizando los datos
# PASO 7 — Normalizar variables predictoras
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Ajustar el scaler solo con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transformar los datos de prueba usando el mismo scaler
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5]


array([[-0.26418592,  1.31853845],
       [ 1.77010585,  2.58795259],
       [ 0.41391134,  0.04912432],
       [ 0.41391134,  0.04912432],
       [ 0.41391134,  0.04912432]])

In [36]:
# Construccion del modelo neuronal
from tensorflow.keras.utils import to_categorical

# Convertir y_train y y_test a formato categorical (one-hot encoding)
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

# Definir la arquitectura del modelo Neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Crear el modelo
model = Sequential()

# Capa de entrada + primera capa oculta
model.add(Dense(16, activation='relu', input_shape=(2,)))

# Segunda capa oculta
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.2))

# Capa de salida (3 clases)
model.add(Dense(3, activation='softmax'))

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenar el modelo
history = model.fit(
    X_train_scaled, y_train_cat,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    verbose=1
)


C:\Users\Carlos Rios\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5513 - loss: 0.9856 - val_accuracy: 0.5579 - val_loss: 0.9752
Epoch 2/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5526 - loss: 0.9801 - val_accuracy: 0.5579 - val_loss: 0.9746
Epoch 3/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5529 - loss: 0.9787 - val_accuracy: 0.5579 - val_loss: 0.9729
Epoch 4/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5527 - loss: 0.9764 - val_accuracy: 0.5579 - val_loss: 0.9710
Epoch 5/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5532 - loss: 0.9749 - val_accuracy: 0.5599 - val_loss: 0.9691
Epoch 6/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5531 - loss: 0.9746 - val_accuracy: 0.5577 - val_loss: 0.9696
Epoch 7/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5540 - loss: 0.9742 - val_accuracy: 0.5599 - val_loss: 0.9697
Epoch 8/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.5537 - loss: 0.9745 - 

In [ ]:
# Ajustes de las variables de la red neuronal
cols_nn = [
    "montoRango",
    "tipoEntidad",
    "entGrp2",
    "categorías",
    "modalidad",
    "subModalidad",
    "mesDePublicación",
    "añoDePublicación",
    "mesDeAdjudicación",
    "añoDeAdjudicación"
]

df_nn2 = df[cols_nn].copy()
df_nn2.head()

,montoRango,mesDePublicación,añoDePublicación,mesDeAdjudicación,añoDeAdjudicación,tipoEntidad_Administración Central,tipoEntidad_Cooperativas,tipoEntidad_Empresas Públicas (Nacionales y Municipales),"tipoEntidad_Entidades Descentralizadas, Autónomas y de Seguridad Social",tipoEntidad_Fideicomisos con fondos públicos,...,modalidad_Procedimientos Regulados por el artículo 44 LCE (Casos de Excepción),modalidad_Procedimientos regulados por el artículo 54 LCE,subModalidad_Adquisiciones en el Extranjero (Art. 44 inciso c),subModalidad_Adquisición por convenios del MSPAS o IGSS (Art. 44 inciso b),subModalidad_Contratación de Servicios Básicos (Art. 44 inciso g),subModalidad_Contratación de Servicios Técnicos y Profesionales individuales (Art. 44 inciso e),subModalidad_Contratos y Convenios que no provengan de procedimientos que determine la LCE,subModalidad_Otras Leyes,"subModalidad_Por Reformas a la LCE, que se origen de otras leyes vigentes",subModalidad_Resolución Judicial
0,Bajo,AGOSTO,2025,AGOSTO,2025,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,Medio,AGOSTO,2025,AGOSTO,2025,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,Bajo,JULIO,2025,AGOSTO,2025,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,Alto,ABRIL,2025,ABRIL,2025,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Alto,JULIO,2025,AGOSTO,2025,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
# Limpieza de datos
df_nn2.isnull().sum()

montoRango                0
tipoEntidad               0
entGrp2                   0
categorías             1402
modalidad                 0
subModalidad         200973
mesDePublicación          0
añoDePublicación          0
mesDeAdjudicación         0
añoDeAdjudicación         0
dtype: int64

In [40]:
# Traatamiento de valores nulos
df_nn2["categorías"] = df_nn2["categorías"].fillna("Desconocido")
df_nn2["subModalidad"] = df_nn2["subModalidad"].fillna("Desconocido")

# Verificacion
df_nn2.isnull().sum()


montoRango           0
tipoEntidad          0
entGrp2              0
categorías           0
modalidad            0
subModalidad         0
mesDePublicación     0
añoDePublicación     0
mesDeAdjudicación    0
añoDeAdjudicación    0
dtype: int64

In [41]:
# Codificar las variables categóricas
from sklearn.preprocessing import LabelEncoder

# Crear copia para trabajar
df_nn3 = df_nn2.copy()

# Crear codificadores
le_tipoEntidad = LabelEncoder()
le_entGrp2 = LabelEncoder()
le_categorias = LabelEncoder()
le_modalidad = LabelEncoder()
le_subModalidad = LabelEncoder()

# Aplicar transformaciones
df_nn3["tipoEntidad"] = le_tipoEntidad.fit_transform(df_nn3["tipoEntidad"])
df_nn3["entGrp2"] = le_entGrp2.fit_transform(df_nn3["entGrp2"])
df_nn3["categorías"] = le_categorias.fit_transform(df_nn3["categorías"])
df_nn3["modalidad"] = le_modalidad.fit_transform(df_nn3["modalidad"])
df_nn3["subModalidad"] = le_subModalidad.fit_transform(df_nn3["subModalidad"])

df_nn3.head()


,montoRango,tipoEntidad,entGrp2,categorías,modalidad,subModalidad,mesDePublicación,añoDePublicación,mesDeAdjudicación,añoDeAdjudicación
0,Bajo,3,1,331,6,5,AGOSTO,2025,AGOSTO,2025
1,Medio,3,1,331,6,5,AGOSTO,2025,AGOSTO,2025
2,Bajo,3,1,331,6,5,JULIO,2025,AGOSTO,2025
3,Alto,5,3,345,13,5,ABRIL,2025,ABRIL,2025
4,Alto,3,1,331,6,5,JULIO,2025,AGOSTO,2025


In [42]:
# Ajustando variable montoRango
from sklearn.preprocessing import LabelEncoder

le_monto = LabelEncoder()
df_nn3["montoRango"] = le_monto.fit_transform(df_nn3["montoRango"])

df_nn3[["montoRango"]].head()

# Seleccionando las variables predictoras y la variable objetivo
X = df_nn3[[
    "tipoEntidad",
    "entGrp2",
    "categorías",
    "modalidad",
    "subModalidad"
]]

y = df_nn3["montoRango"]

# Verificacion
X.head(), y.head()


(   tipoEntidad  entGrp2  categorías  modalidad  subModalidad
 0            3        1         331          6             5
 1            3        1         331          6             5
 2            3        1         331          6             5
 3            5        3         345         13             5
 4            3        1         331          6             5,
 0    1
 1    2
 2    1
 3    0
 4    0
 Name: montoRango, dtype: int64)

In [43]:
# Definicion del train y test sets
# PASO 6 — Separar en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.20,     # 20% prueba
    random_state = 42,    # reproducible
    stratify = y          # mantiene proporciones de clases
)

# Ver tamaños
X_train.shape, X_test.shape


((166751, 5), (41688, 5))

In [44]:
# Normalizando los datos
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Ajustar solo con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transformar el conjunto de prueba con el scaler ya entrenado
X_test_scaled = scaler.transform(X_test)

# Mostrar primeras filas escaladas para verificar
X_train_scaled[:5]


array([[-0.26418592,  1.31853845,  0.38359371,  4.12608877, -3.07093079],
       [ 1.77010585,  2.58795259,  0.22389635, -0.26074114,  0.01781925],
       [ 0.41391134,  0.04912432,  0.47484935, -0.26074114,  0.01781925],
       [ 0.41391134,  0.04912432,  0.46344239, -0.26074114,  0.01781925],
       [ 0.41391134,  0.04912432, -1.47573989, -0.26074114,  0.01781925]])

In [45]:
# Construccion de la red neuronal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Crear el modelo
modelo_nn = Sequential([
    Dense(16, activation='relu', input_shape=(5,)),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')   # 3 clases: Bajo, Medio, Alto
])

# Compilar el modelo
modelo_nn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

modelo_nn.summary()


C:\Users\Carlos Rios\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 259 (1.01 KB)

 Trainable params: 259 (1.01 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
# Entrenando el modelo
historial = modelo_nn.fit(
    X_train_scaled,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5678 - loss: 0.9422 - val_accuracy: 0.5733 - val_loss: 0.9220
Epoch 2/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5727 - loss: 0.9248 - val_accuracy: 0.5771 - val_loss: 0.9188
Epoch 3/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5743 - loss: 0.9226 - val_accuracy: 0.5766 - val_loss: 0.9180
Epoch 4/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5746 - loss: 0.9212 - val_accuracy: 0.5774 - val_loss: 0.9158
Epoch 5/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5749 - loss: 0.9201 - val_accuracy: 0.5774 - val_loss: 0.9181
Epoch 6/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5751 - loss: 0.9191 - val_accuracy: 0.5774 - val_loss: 0.9143
Epoch 7/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5754 - loss: 0.9183 - val_accuracy: 0.5776 - val_loss: 0.9133
Epoch 8/20
4169/4169 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.5754 - loss: 0.9174 - 

In [47]:
# Evaluando el modelo en el conjunto de prueba
loss_test, acc_test = modelo_nn.evaluate(X_test_scaled, y_test, verbose=1)

print("🔹 Pérdida en test:", loss_test)
print("🔹 Exactitud (accuracy) en test:", acc_test)


1303/1303 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5794 - loss: 0.9064
🔹 Pérdida en test: 0.9064416289329529
🔹 Exactitud (accuracy) en test: 0.5794233083724976


In [48]:
# Matriz de confusión
from sklearn.metrics import confusion_matrix, classification_report

y_pred = modelo_nn.predict(X_test_scaled).argmax(axis=1)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


1303/1303 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
[[ 2907   206  7204]
 [   81  1668  8673]
 [  490   879 19580]]
              precision    recall  f1-score   support

           0       0.84      0.28      0.42     10317
           1       0.61      0.16      0.25     10422
           2       0.55      0.93      0.69     20949

    accuracy                           0.58     41688
   macro avg       0.66      0.46      0.46     41688
weighted avg       0.64      0.58      0.52     41688



In [61]:
# Comparativa ARBOL A
# PASO 1 — Crear dataset del Modelo A
df_A = df_nn2[[
    "tipoEntidad",
    "categorías",
    "modalidad",
    "subModalidad",
    "entGrp2"   # objetivo
]].copy()

df_A.head()



,tipoEntidad,categorías,modalidad,subModalidad,entGrp2
0,"Entidades Descentralizadas, Autónomas y de Seg...",Salud e insumos hospitalarios,Compra Directa con Oferta Electrónica (Art. 43...,Desconocido,Descentralizada
1,"Entidades Descentralizadas, Autónomas y de Seg...",Salud e insumos hospitalarios,Compra Directa con Oferta Electrónica (Art. 43...,Desconocido,Descentralizada
2,"Entidades Descentralizadas, Autónomas y de Seg...",Salud e insumos hospitalarios,Compra Directa con Oferta Electrónica (Art. 43...,Desconocido,Descentralizada
3,"Gobiernos Locales (Municipalidades, Mancomunid...","Seguros, fianzas y servicios bancarios",Negociaciones entre Entidades Públicas (Art. 2...,Desconocido,Municipal
4,"Entidades Descentralizadas, Autónomas y de Seg...",Salud e insumos hospitalarios,Compra Directa con Oferta Electrónica (Art. 43...,Desconocido,Descentralizada


In [62]:
# Codificar variables categóricas
from sklearn.preprocessing import LabelEncoder

# Crear copia para no modificar df_A original
df_A_encoded = df_A.copy()

# Crear codificadores
le_tipoEntidad = LabelEncoder()
le_categorias = LabelEncoder()
le_modalidad = LabelEncoder()
le_subModalidad = LabelEncoder()
le_entGrp2 = LabelEncoder()

# Aplicar codificación
df_A_encoded["tipoEntidad"] = le_tipoEntidad.fit_transform(df_A_encoded["tipoEntidad"])
df_A_encoded["categorías"] = le_categorias.fit_transform(df_A_encoded["categorías"])
df_A_encoded["modalidad"] = le_modalidad.fit_transform(df_A_encoded["modalidad"])
df_A_encoded["subModalidad"] = le_subModalidad.fit_transform(df_A_encoded["subModalidad"])

# Objetivo
df_A_encoded["entGrp2"] = le_entGrp2.fit_transform(df_A_encoded["entGrp2"])

df_A_encoded.head()



,tipoEntidad,categorías,modalidad,subModalidad,entGrp2
0,3,331,6,5,1
1,3,331,6,5,1
2,3,331,6,5,1
3,5,345,13,5,3
4,3,331,6,5,1


In [ ]:
# Definicion del train y test sets
from sklearn.model_selection import train_test_split

# Separar X y y
X_A = df_A_encoded[["tipoEntidad", "categorías", "modalidad", "subModalidad"]]
y_A = df_A_encoded["entGrp2"]

# Dividir en train / test
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(
    X_A, y_A, test_size=0.20, random_state=42, stratify=y_A
)

X_train_A.shape, X_test_A.shape



((166751, 4), (41688, 4))

In [65]:
# Escalar los datos
from sklearn.preprocessing import StandardScaler

scaler_A = StandardScaler()

# Ajustamos solo con X_train (buena práctica)
X_train_A_scaled = scaler_A.fit_transform(X_train_A)

# Transformamos X_test con el mismo scaler
X_test_A_scaled = scaler_A.transform(X_test_A)

X_train_A_scaled[:5]


array([[-1.62218474,  0.47572874, -0.26117112,  0.01566514],
       [-1.62218474, -1.47182124, -0.26117112,  0.01566514],
       [ 1.76864748, -3.3054794 , -0.26117112,  0.01566514],
       [-1.62218474, -3.3054794 , -0.26117112,  0.01566514],
       [ 0.41231459,  0.46433956, -0.26117112,  0.01566514]])

In [66]:
# Definir la red neuronal del Modelo A  
from keras.models import Sequential
from keras.layers import Dense

# Número de clases en entGrp2
num_clases_A = len(y_A.unique())

# Crear modelo
modelo_A = Sequential([
    Dense(16, activation='relu', input_shape=(X_train_A_scaled.shape[1],)),
    Dense(8, activation='relu'),
    Dense(num_clases_A, activation='softmax')
])

# Compilar modelo
modelo_A.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

modelo_A.summary()


C:\Users\Carlos Rios\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │            54 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 270 (1.05 KB)

 Trainable params: 270 (1.05 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
# Entrenando el Modelo A
hist_A = modelo_A.fit(
    X_train_A_scaled, 
    y_train_A,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)



Epoch 1/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9984 - loss: 0.0082 - val_accuracy: 0.9993 - val_loss: 0.0054
Epoch 2/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9994 - loss: 0.0035 - val_accuracy: 0.9997 - val_loss: 0.0031
Epoch 3/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9997 - loss: 0.0020 - val_accuracy: 0.9998 - val_loss: 0.0022
Epoch 4/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9998 - loss: 0.0014 - val_accuracy: 0.9998 - val_loss: 0.0019
Epoch 5/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9999 - loss: 0.0012 - val_accuracy: 0.9999 - val_loss: 0.0017
Epoch 6/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9999 - loss: 0.0010 - val_accuracy: 0.9999 - val_loss: 0.0016
Epoch 7/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9999 - loss: 9.0751e-04 - val_accuracy: 0.9999 - val_loss: 0.0015
Epoch 8/20
2085/2085 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9999 - loss: 7.892

In [69]:
# Evaluar el modelo en test

loss_A_test, acc_A_test = modelo_A.evaluate(X_test_A_scaled, y_test_A, verbose=1)
print("Pérdida en test:", loss_A_test)
print("Exactitud en test:", acc_A_test)


1303/1303 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 2.4667e-06
Pérdida en test: 2.466745399942738e-06
Exactitud en test: 1.0


In [73]:
# Matriz de confusión y reporte de clasificación

from sklearn.metrics import confusion_matrix, classification_report

# Predicciones del modelo
y_pred_A = modelo_A.predict(X_test_A_scaled).argmax(axis=1)

# Matriz de confusión
print(confusion_matrix(y_test_A, y_pred_A))

# Reporte de clasificación
print(classification_report(y_test_A, y_pred_A))


1303/1303 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
[[10037     0     0     0     0     0]
 [    0 26614     0     0     0     0]
 [    0     0  1837     0     0     0]
 [    0     0     0  2963     0     0]
 [    0     0     0     0   217     0]
 [    0     0     0     0     0    20]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10037
           1       1.00      1.00      1.00     26614
           2       1.00      1.00      1.00      1837
           3       1.00      1.00      1.00      2963
           4       1.00      1.00      1.00       217
           5       1.00      1.00      1.00        20

    accuracy                           1.00     41688
   macro avg       1.00      1.00      1.00     41688
weighted avg       1.00      1.00      1.00     41688



In [74]:
# Ejecucion del caso hipotetico Arbol A

# Caso hipotético
nuevo_df = pd.DataFrame({
    'tipoEntidad': ["Entidades Descentralizadas, Autónomas y de Seguridad Social"],
    'entGrp2': ["Descentralizada"],
    'categorías': ["Salud e insumos hospitalarios"],
    'modalidad': ["Compra Directa con Oferta Electrónica (Art. 43 Ley)"],
    'subModalidad': ["Desconocido"]
})

# One-hot encoding del nuevo caso
nuevo_df_encoded = pd.get_dummies(nuevo_df)

# Alinear columnas con las usadas por X_train_A
nuevo_df_encoded = nuevo_df_encoded.reindex(columns=X_train_A.columns, fill_value=0)

nuevo_df_encoded

nuevo_scaled = scaler_A.transform(nuevo_df_encoded)
prediccion_A = modelo_A.predict(nuevo_scaled).argmax(axis=1)[0]
prediccion_A

mapa_clases = {0: "Bajo", 1: "Medio", 2: "Alto"}
resultado = mapa_clases[prediccion_A]
print("Predicción final del caso hipotético:", resultado)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicción final del caso hipotético: Bajo


In [77]:
# Escenario 2. ARBOL B

# Contar frecuencia por clase
freq = df_nn2["subModalidad"].value_counts()

freq
# Remover subModalidad con frecuencia 1
df_B = df_nn2[df_nn2["subModalidad"] != "Resolución Judicial"].copy()

# Verificación
df_B["subModalidad"].value_counts()


subModalidad
Desconocido                                                                           200973
Otras Leyes                                                                             4031
Contratos y Convenios que no provengan de procedimientos que determine la LCE           2266
Adquisiciones en el Extranjero (Art. 44 inciso c)                                        370
Por Reformas a la LCE, que se origen de otras leyes vigentes                             268
Contratación de Servicios Técnicos y Profesionales individuales (Art. 44 inciso e)       200
Adquisición por convenios del MSPAS o IGSS (Art. 44 inciso b)                            184
Contratación de Servicios Básicos (Art. 44 inciso g)                                     146
Name: count, dtype: int64

In [96]:
df.columnsdf_B = df[[
    "tipoEntidad",
    "entGrp2",
    "categorías",
    "modalidad",
    "subModalidad",
    "montoRango"
]].copy()

df_B.head()


C:\Users\Carlos Rios\AppData\Local\Temp\ipykernel_3480\3499515834.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnsdf_B = df[[


,entidadCompradora,monto,categorías,tipoEntidad,entGrp,montoRango,entGrp2
0,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,1987.50,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Bajo,Descentralizada
1,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,36000.00,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Medio,Descentralizada
2,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,293.70,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Bajo,Descentralizada
3,"MUNICIPALIDAD DE SAN JOSÉ PINULA, GUATEMALA",222223.47,"Seguros, fianzas y servicios bancarios","Gobiernos Locales (Municipalidades, Mancomunid...",Otra,Alto,Municipal
4,INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-,90000.00,Salud e insumos hospitalarios,"Entidades Descentralizadas, Autónomas y de Seg...",Descentralizada,Alto,Descentralizada


In [103]:
X_B = df_B.drop("montoRango", axis=1)
y_B = df_B["montoRango"]

X_B_encoded = pd.get_dummies(X_B)
X_B_encoded.head()

X_B_encoded.shape


(208439, 940)

In [ ]:
# PASO 5 — División en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(
    X_B_encoded,
    y_B,
    test_size=0.20,
    random_state=42,
    stratify=y_B
)

X_train_B.shape, X_test_B.shape


((166751, 940), (41688, 940))

In [ ]:
# PASO 6 — Escalar las variables
from sklearn.preprocessing import StandardScaler

scaler_B = StandardScaler()

X_train_B_scaled = scaler_B.fit_transform(X_train_B)
X_test_B_scaled  = scaler_B.transform(X_test_B)


In [ ]:
# PASO 7 — Construir el modelo neuronal para ARBOL B
from keras.models import Sequential
from keras.layers import Dense

modelo_B = Sequential([
    Dense(16, activation="relu", input_shape=(X_train_B_scaled.shape[1],)),
    Dense(8, activation="relu"),
    Dense(3, activation="softmax")  # 3 clases: Bajo, Medio, Alto
])

modelo_B.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

modelo_B.summary()


C:\Users\Carlos Rios\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 16)             │        15,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,219 (59.45 KB)

 Trainable params: 15,219 (59.45 KB)

 Non-trainable params: 0 (0.00 B)

In [111]:
# PASO 8 — Entrenar el modelo

from sklearn.preprocessing import LabelEncoder

le_B = LabelEncoder()
y_train_B_encoded = le_B.fit_transform(y_train_B)
y_test_B_encoded  = le_B.transform(y_test_B)

import numpy as np
np.unique(y_train_B_encoded, return_counts=True)


(array([0, 1, 2]), array([41268, 41690, 83793]))

In [112]:
# PASO 9 - Evaluacion
loss_B, acc_B = modelo_B.evaluate(
    X_test_B_scaled,
    y_test_B_encoded,
    verbose=1
)

loss_B, acc_B


1303/1303 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2654 - loss: 1.1679


(1.1679400205612183, 0.26540011167526245)

In [113]:
# Paso 10 Repote
y_pred_B = modelo_B.predict(X_test_B_scaled).argmax(axis=1)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test_B_encoded, y_pred_B))
print(classification_report(y_test_B_encoded, y_pred_B))


1303/1303 ━━━━━━━━━━━━━━━━━━━━ 1s 940us/step
[[ 4177  4118  2022]
 [ 4706  3928  1788]
 [ 7763 10227  2959]]
              precision    recall  f1-score   support

           0       0.25      0.40      0.31     10317
           1       0.21      0.38      0.27     10422
           2       0.44      0.14      0.21     20949

    accuracy                           0.27     41688
   macro avg       0.30      0.31      0.27     41688
weighted avg       0.34      0.27      0.25     41688

